In [ ]:
# ! pip install vanna[chromadb,openai] -q

In [ ]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore


In [ ]:
openai_api = ''

In [ ]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': openai_api, 'model': 'gpt-4o'})

In [ ]:
vn.connect_to_sqlite('QRM.db')

In [ ]:
# New way to train the Question and SQL pair 
# 
vn.train(
    question="What is the average age of our customers?", 
    sql="SELECT AVG(age) FROM customers"
)

In [ ]:
vn.generate_sql("What are the top 10 customers by sales?")

# Above function will call following functions

# get_similar_question_sql

# get_related_ddl

# get_related_documentation

# get_sql_prompt

# submit_prompt

In [ ]:
# For run the generated sql query

vn.run_sql("SELECT * FROM my_table")

In [ ]:
# Will generate the plotly code
# Need R&D
vn.generate_plotly_code()

In [ ]:
# Aboves code output is input for this function


fig = vn.get_plotly_figure(
    plotly_code="fig = px.bar(df, x='name', y='salary')",
    df=df
)
fig.show()